# Gossipcop

In [ ]:
import torch

input_ids_gossipcop = torch.load('/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/input_ids_gossipcop_users')
attention_masks_gossipcop = torch.load('/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/attention_masks_gossipcop_users')
labels_gossipcop = torch.load('/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/labels_gossipcop_users')

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_gossipcop, attention_masks_gossipcop, labels_gossipcop)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, valid_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


# For validation the order doesn't matter, so we'll just read them sequentially.
valid_loader = DataLoader(
            valid_dataset, # The validation samples.
            sampler = SequentialSampler(valid_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



print('Train = %d (%d) | Validation = %d (%d)' %(len(train_dataset), len(train_loader), len(valid_dataset), len(valid_loader)))

28,993 training samples
3,222 validation samples
Train = 28993 (1813) | Validation = 3222 (202)


In [ ]:
# !pip freeze | grep transformers
!pip install transformers=="2.2.0"

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    model = model.to(device)

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

def flat_precision(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return recall_score(labels_flat, pred_flat)

def flat_recall(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return precision_score(labels_flat, pred_flat)

def flat_f1_score(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)

In [ ]:
import random
import numpy as np
from tqdm import tqdm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

sentence_embedding_train = []
sentence_embedding_val = []


# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits, hidden_states_train = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
#         token_vecs = hidden_states_train[-2][0]

        # Calculate the average of all 22 token vectors.
#         sentence_embedding_train.append(torch.mean(token_vecs, dim=0))
        
        

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
#         print(loss)
#         print(type(loss))
        total_train_loss += loss.item()
        

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
    
    

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_loader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_precision = 0
    total_eval_recall = 0
    total_eval_f1 = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in valid_loader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits, hidden_states_val) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        
#         token_vecs = hidden_states_val[-2][0]

            # Calculate the average of all 22 token vectors.
#         sentence_embedding_val.append(torch.mean(token_vecs, dim=0))
            
        # Accumulate the validation loss.
#         print(loss)
#         print(type(loss))
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        total_eval_precision += flat_precision(logits, label_ids)
        total_eval_recall += flat_recall(logits, label_ids)
        total_eval_f1 += flat_f1_score(logits, label_ids)
       

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(valid_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    
    avg_val_precision = total_eval_precision / len(valid_loader)
    avg_val_recall = total_eval_recall / len(valid_loader)
    avg_val_f1 = total_eval_f1 / len(valid_loader)

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(valid_loader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time,
            'Valid. Prec': avg_val_precision,
            'Valid. Recall': avg_val_recall,
            'Valid. F1': avg_val_f1,
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    50  of  1,813.    Elapsed: 0:00:16.
  Batch   100  of  1,813.    Elapsed: 0:00:31.
  Batch   150  of  1,813.    Elapsed: 0:00:47.
  Batch   200  of  1,813.    Elapsed: 0:01:02.
  Batch   250  of  1,813.    Elapsed: 0:01:18.
  Batch   300  of  1,813.    Elapsed: 0:01:34.
  Batch   350  of  1,813.    Elapsed: 0:01:50.
  Batch   400  of  1,813.    Elapsed: 0:02:06.
  Batch   450  of  1,813.    Elapsed: 0:02:22.
  Batch   500  of  1,813.    Elapsed: 0:02:38.
  Batch   550  of  1,813.    Elapsed: 0:02:54.
  Batch   600  of  1,813.    Elapsed: 0:03:10.
  Batch   650  of  1,813.    Elapsed: 0:03:25.
  Batch   700  of  1,813.    Elapsed: 0:03:41.
  Batch   750  of  1,813.    Elapsed: 0:03:57.
  Batch   800  of  1,813.    Elapsed: 0:04:13.
  Batch   850  of  1,813.    Elapsed: 0:04:29.
  Batch   900  of  1,813.    Elapsed: 0:04:45.
  Batch   950  of  1,813.    Elapsed: 0:05:01.
  Batch 1,000  of  1,813.    Elapsed: 0:05:17.
  Batch 1,050  of

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time,Valid. Prec,Valid. Recall,Valid. F1
epoch,,,,,,,,
1,0.21,0.20,0.92,0:09:35,0:00:20,0.98,0.93,0.95
2,0.19,0.19,0.91,0:09:38,0:00:20,0.97,0.94,0.95
3,0.17,0.23,0.90,0:09:40,0:00:20,0.95,0.94,0.94
4,0.15,0.26,0.89,0:09:36,0:00:20,0.93,0.95,0.94


In [ ]:
print(np.mean(df_stats['Valid. Accur.']))
print(np.mean(df_stats['Valid. Prec']))
print(np.mean(df_stats['Valid. Recall']))
print(np.mean(df_stats['Valid. F1']))

0.903697400990099
0.95817528696551
0.9379749066939664
0.946140450071429


In [ ]:
import os
if not os.path.exists('/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/'+'sent_emb_gossipcop.pt'):
    sentence_embedding = []
    model.eval()

    for i in tqdm(range(len(input_ids_gossipcop))):


        input_ids_gossipcop = input_ids_gossipcop.to(device)
        a=torch.reshape(input_ids_gossipcop[i], (1,len(input_ids_gossipcop[i])))

        with torch.no_grad():

            outputs = model(a, token_type_ids=None)
            # Evaluating the model will return a different number of objects based on 
            # how it's  configured in the `from_pretrained` call earlier. In this case, 
            # becase we set `output_hidden_states = True`, the third item will be the 
            # hidden states from all layers. See the documentation for more details:
            # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
            (logits, hidden_states) = outputs
        token_vecs = hidden_states[-2][0]

        # Calculate the average of all 22 token vectors.
        sentence_embedding.append(torch.mean(token_vecs, dim=0))
    torch.save(sentence_embedding, '/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/'+'sent_emb_gossipcop')
else:
    sentence_embedding = torch.load('/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/'+'sent_emb_gossipcop.pt')




NameError: name 'model' is not defined

In [ ]:
import torch
sentence_embedding = torch.load('/home/local/ASUAD/mkarami/Documents/Motivational Factors/cache/'+'sent_emb_gossipcop')

In [ ]:
import os
base_dir = os.getcwd() + '/'
print('Base Dir:', base_dir)

import pickle
features_gossipcop_fake_users = pickle.load(open(base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_gossipcop_fake_users.pkl', 'rb'))
features_gossipcop_fake_users = features_gossipcop_fake_users.values()
features_gossipcop_real_users = pickle.load(open(base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_gossipcop_real_users.pkl', 'rb'))
features_gossipcop_real_users = features_gossipcop_real_users.values()

Base Dir: /home/local/ASUAD/mkarami/Documents/Motivational Factors/


In [ ]:
gossipcop_features = []
for i in range(len(list(features_gossipcop_fake_users))):
    gossipcop_features.append(list(list(features_gossipcop_fake_users)[i].values()))
for i in range(len(list(features_gossipcop_real_users))):
    gossipcop_features.append(list(list(features_gossipcop_real_users)[i].values()))
gossipcop_features = [[v if v is not None else 0 for v in nested] for nested in gossipcop_features]


In [ ]:
X_Gossipcop_concated_ver = []
for i in range(len(gossipcop_features)):
    X_Gossipcop_concated_ver.append(gossipcop_features[i]+sentence_embedding[i].tolist())

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_Gossipcop_concated_ver, labels_gossipcop.tolist(), test_size=0.33, random_state=42)
clf = MLPClassifier(hidden_layer_sizes=(1000,500))
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(1000, 500))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.9269118615370144
0.9637055315648776
0.9567214453404104
0.9602007888131947
